### 1. 데이터 준비

In [134]:
# 모듈 로딩 및 데이터 확인

import pandas as pd

df = pd.read_csv('repair2.csv')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13262 entries, 0 to 13261
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   caseID               13262 non-null  int64  
 1   taskID               13260 non-null  object 
 2   originator           13260 non-null  object 
 3   eventtype            13257 non-null  object 
 4   contact              1000 non-null   object 
 5   RepairType           927 non-null    object 
 6   objectKey            927 non-null    float64
 7   RepairInternally     927 non-null    object 
 8   EstimatedRepairTime  927 non-null    float64
 9   RepairCode           927 non-null    float64
 10  RepairOK             1854 non-null   object 
 11  date                 13249 non-null  object 
 12  time                 13249 non-null  object 
dtypes: float64(3), int64(1), object(9)
memory usage: 1.3+ MB


,caseID,taskID,originator,eventtype,contact,RepairType,objectKey,RepairInternally,EstimatedRepairTime,RepairCode,RepairOK,date,time
0,419,InformClientSurvey,System,complete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-06,11:51
1,561,ArrangeSurvey,Monica,complete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-07,13:48
2,902,ReadyInformClient,System,complete,NaN,NaN,NaN,NaN,NaN,NaN,True,2020-01-08,16:20
3,18,Survey,Edd,start,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-09,6:55
4,852,Survey,Edd,complete,NaN,NaN,NaN,True,240.0,1.0,NaN,2020-01-10,11:33


- contact, RepairKey, objectKey, RepairInternally, EstimatedRepairTime, RepairCode, RepairOk에 결측치 많음



o   caseID: 집 수리 요청 접수 번호

o   taskID: 업무

o   originator: 업무 담당자

o   eventtype: 업무의 시작(start)과 끝(complete)을 표시. 

o   contact: 집 수리 요청의 요청 채널.

o   RepairType: 수리 방식.

o   objectKey: 수리 대상 집 key.

o   RepairInternally: 내부 수리 여부.

o   EstimatedRepairTime: 예상 수리 시간.

o   RepairCode: 수리 종류.

o   RepairOK: 수리 정상 종료 여부

o   Date: 업무 수행 일자

o   Time: 업무 수행 시각

In [135]:
# 날짜와 시간이 동시에 결측치인 컬럼이 13개이므로 둘 중 하나만 결측치인 행은 없음
len(df[(df['date'].isna() == True) & (df['time'].isna() == True)])

13

In [136]:
# date와 time을 하나의 컬럼으로 변환
df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
df.head()

,caseID,taskID,originator,eventtype,contact,RepairType,objectKey,RepairInternally,EstimatedRepairTime,RepairCode,RepairOK,date,time,datetime
0,419,InformClientSurvey,System,complete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-06,11:51,2020-01-06 11:51:00
1,561,ArrangeSurvey,Monica,complete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-07,13:48,2020-01-07 13:48:00
2,902,ReadyInformClient,System,complete,NaN,NaN,NaN,NaN,NaN,NaN,True,2020-01-08,16:20,2020-01-08 16:20:00
3,18,Survey,Edd,start,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-09,6:55,2020-01-09 06:55:00
4,852,Survey,Edd,complete,NaN,NaN,NaN,True,240.0,1.0,NaN,2020-01-10,11:33,2020-01-10 11:33:00


In [ ]:
df['']

### 2. 결측치가 많은 컬럼 탐색

#### 2-1 contact 컬럼

In [137]:
# taskID가 FirstContact일때만 contact 컬럼이 존재하는 것으로 보임
print('-'*7 + 'contact가 결측 O' + '-'*7)
print(df[df['contact'].isna() == True]['taskID'].value_counts())
# contact가 결측치인 행에는 taskID에 FirstContact가 없다

print('-'*7 + 'contact가 결측 X' + '-'*7)

print(df[df['contact'].isna() == False]['taskID'].value_counts())
# contact가 결측이 아닌 행에는 taskID가 모두 FirstContact이다
# 즉, FirstContact일때만 contact값이 존재한다

-------contact가 결측 O-------
taskID
ArrangeSurvey             1854
Survey                    1854
MakeTicket                1854
InternRepair              1094
ReadyInformClient          927
TicketReady                927
RepairReady                927
SendTicketToFinAdmin       927
InformClientSurvey         926
ImmediateRepair            766
ExternRepair               132
InformClientWrongPlace      73
Name: count, dtype: int64
-------contact가 결측 X-------
taskID
FirstContact    999
Name: count, dtype: int64


#### 2-2 RepairType, objectKey 컬럼

In [138]:
# RepairType과 objectKey가 항상 함께 존재하는 것 처럼 보임
len(df[(df['RepairType'].isna() == False) & (df['objectKey'].isna() == False)])
# 길이가 927이므로 항상 같이 존재

927

In [139]:
# taskID가 MakeTicket일때만 RepairType과 objectKey가 존재하는 것으로 보임
df[(df['RepairType'].isna() == True) & (df['objectKey'].isna() == True)]['taskID'].value_counts()
# 하지만 RepairType과 objectKey가 결측치일때도 MakeTicket 값이 존재함

taskID
ArrangeSurvey             1854
Survey                    1854
InternRepair              1094
FirstContact               999
ReadyInformClient          927
TicketReady                927
MakeTicket                 927
RepairReady                927
SendTicketToFinAdmin       927
InformClientSurvey         926
ImmediateRepair            766
ExternRepair               132
InformClientWrongPlace      73
Name: count, dtype: int64

In [140]:
# eventtype이 complete일때의 조건도 넣어서 다시 확인
mask1 = (df['RepairType'].isna() == True)
mask2 = (df['objectKey'].isna() == True)
mask3 = (df['eventtype'] == 'complete')
print(df[mask1 & mask2 & mask3]['taskID'].value_counts())
# MakeTicket 값이 존재하지 않음
print()
mask1 = (df['RepairType'].isna() == False)
mask2 = (df['objectKey'].isna() == False)
mask3 = (df['eventtype'] == 'complete')
print(df[mask1 & mask2 & mask3]['taskID'].value_counts())
# 즉, taskID가 MakeTicket이면서 eventtype이 complete일때만 RepairType과 objectKey값이 함께 존재

taskID
FirstContact              999
ArrangeSurvey             927
ReadyInformClient         927
RepairReady               927
SendTicketToFinAdmin      927
Survey                    926
TicketReady               926
InformClientSurvey        925
InternRepair              547
ImmediateRepair           383
InformClientWrongPlace     73
Name: count, dtype: int64

taskID
MakeTicket    927
Name: count, dtype: int64


In [141]:
df['objectKey'].value_counts()

objectKey
5997.0    3
8615.0    3
4118.0    2
3351.0    2
2091.0    2
         ..
3807.0    1
2296.0    1
8266.0    1
4370.0    1
9732.0    1
Name: count, Length: 890, dtype: int64

In [146]:
df[df['taskID'] == 'MakeTicket'].head()

,caseID,taskID,originator,eventtype,contact,RepairType,objectKey,RepairInternally,EstimatedRepairTime,RepairCode,RepairOK,date,time,datetime
6,521,MakeTicket,Dian,complete,NaN,E,13.0,NaN,NaN,NaN,NaN,2020-01-12,20:30,2020-01-12 20:30:00
8,604,MakeTicket,Monica,start,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-14,18:37,2020-01-14 18:37:00
22,897,MakeTicket,Dian,start,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-28,4:09,2020-01-28 04:09:00
27,815,MakeTicket,Monica,start,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-02,16:40,2020-02-02 16:40:00
31,928,MakeTicket,Monica,start,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-06,22:19,2020-02-06 22:19:00


In [143]:
# RepairOK 컬럼의 경우 TRUE 이외의 값은 보이지 않는데 담당자가 System이면서 complete가 된 경우에 채워진 것으로 보임
# 또한, taskID의 종류가 RepairReady나 ReadyInformClient일때 RepairOK 값이 존재하는 것으로 보임
df['RepairOK'].value_counts() # True 값만 존재

RepairOK
True    1854
Name: count, dtype: int64

In [144]:
df[df['RepairOK'] == True][['taskID', 'originator', 'eventtype']].value_counts()
# RepairOK 컬럼은 시스템이 ReadyInformClient, RepairReady 태스크가 끝날때 True를 채우는 것으로 보임

taskID             originator  eventtype
ReadyInformClient  System      complete     927
RepairReady        System      complete     927
Name: count, dtype: int64

In [145]:
df['caseID'].value_counts()

caseID
865    18
512    18
775    17
653    17
456    17
       ..
213     2
145     2
766     2
918     2
312     2
Name: count, Length: 1000, dtype: int64